<a href="https://colab.research.google.com/github/bhav380-2/facialExpressionRecognition/blob/main/facialRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git


Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 30.22 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [ ]:
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)

  for row in reader:
    key = row[2].lower()
    if key not in data:
      data[key] = []
    data[key].append(row[1])

emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
len(data['happiness'])

5696

In [ ]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [21]:
# for emotion in emotion_list:
#   os.mkdir(os.path.join('master_data/training/',emotion))
#   os.mkdir(os.path.join('master_data/testing/',emotion))

In [ ]:
from shutil import copyfile
split_size = 0.8

for emotion, images in data.items():

  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]


  for image in train_images:
    source = os.path.join('/content/facial_expressions/images',image)
    print(source)
    dest = os.path.join('master_data/training',emotion,image)
    print(dest)
    copyfile(source, dest)


  for image in test_images:
    source = os.path.join('/content/facial_expressions/images',image)
    dest = os.path.join('master_data/testing',emotion,image)
    copyfile(source,dest)


Streaming output truncated to the last 5000 lines.
/content/facial_expressions/images/Bill_Byrne_0001.jpg
master_data/training/neutral/Bill_Byrne_0001.jpg
/content/facial_expressions/images/Bill_Callahan_0001.jpg
master_data/training/neutral/Bill_Callahan_0001.jpg
/content/facial_expressions/images/Bill_Callahan_0002.jpg
master_data/training/neutral/Bill_Callahan_0002.jpg
/content/facial_expressions/images/Bill_Callahan_0003.jpg
master_data/training/neutral/Bill_Callahan_0003.jpg
/content/facial_expressions/images/Bill_Cartwright_0001.jpg
master_data/training/neutral/Bill_Cartwright_0001.jpg
/content/facial_expressions/images/Bill_Clancy_0001.jpg
master_data/training/neutral/Bill_Clancy_0001.jpg
/content/facial_expressions/images/Bill_Clinton_0001.jpg
master_data/training/neutral/Bill_Clinton_0001.jpg
/content/facial_expressions/images/Bill_Clinton_0003.jpg
master_data/training/neutral/Bill_Clinton_0003.jpg
/content/facial_expressions/images/Bill_Clinton_0006.jpg
master_data/training/n

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [17]:
model = tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
    MaxPooling2D(2,2),
     Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(2,2),
     Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024,activation='relu'),
    Dense(8,activation='softmax')

    ])

model.compile(optimizer=Adam(learning_rate=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 49, 49, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 10, 10, 64)       

In [18]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)


test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [19]:
es = EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

In [20]:
model.fit(train_generator,epochs=10,verbose=1,validation_data=test_generator,callbacks=[es])

Epoch 1/10
86/86 [==============================] - 17s 179ms/step - loss: 1.1972 - acc: 0.4777 - val_loss: 1.0199 - val_acc: 0.5011
Epoch 2/10
86/86 [==============================] - 16s 191ms/step - loss: 1.0037 - acc: 0.5569 - val_loss: 1.0087 - val_acc: 0.5635
Epoch 3/10
86/86 [==============================] - 15s 180ms/step - loss: 0.8344 - acc: 0.6934 - val_loss: 0.9394 - val_acc: 0.6313
Epoch 4/10
86/86 [==============================] - 15s 173ms/step - loss: 0.7354 - acc: 0.7415 - val_loss: 0.9582 - val_acc: 0.6327
Epoch 5/10
86/86 [==============================] - 18s 209ms/step - loss: 0.6887 - acc: 0.7613 - val_loss: 1.1951 - val_acc: 0.6291
